# Generative networks

Recurrent Neural Networks (RNNs) နှင့် Long Short Term Memory Cells (LSTMs) နှင့် Gated Recurrent Units (GRUs) ကဲ့သို့သော gated cell များသည် ဘာသာစကားမော်ဒယ်တစ်ခုအတွက် စနစ်တစ်ခုကို ပံ့ပိုးပေးနိုင်သည်။ အဆိုပါစနစ်သည် စကားလုံးများ၏ အစီအစဉ်ကို သင်ယူနိုင်ပြီး အစီအစဉ်အတွင်း နောက်တစ်ခုထွက်မည့် စကားလုံးကို ခန့်မှန်းပေးနိုင်သည်။ ဒီလိုဖြစ်တာကြောင့် RNNs ကို **ဖန်တီးမှုဆိုင်ရာလုပ်ငန်းများ** (ဥပမာ - ပုံမှန်စာသားဖန်တီးခြင်း၊ စက်ဖြင့်ဘာသာပြန်ခြင်း၊ ပုံဖော်ရေးသားခြင်း) အတွက် အသုံးပြုနိုင်သည်။

ယခင်ယူနစ်တွင် ဆွေးနွေးခဲ့သည့် RNN architecture တွင် RNN unit တစ်ခုစီသည် နောက်ထွက်မည့် hidden state ကို output အဖြစ် ထုတ်ပေးသည်။ သို့သော် RNN unit တစ်ခုစီတွင် output တစ်ခုထပ်ထည့်နိုင်ပြီး၊ အဲဒီ output က **sequence** (မူရင်း sequence နှင့် အရှည်တူ) ကို ထုတ်ပေးနိုင်သည်။ ထို့အပြင်၊ RNN units များကို အဆင့်တစ်ခုစီတွင် input မလိုအပ်ဘဲ အသုံးပြုနိုင်ပြီး၊ စတင်အခြေအနေ vector တစ်ခုသာ လက်ခံပြီး output sequence တစ်ခုကို ထုတ်ပေးနိုင်သည်။

ဒီ notebook တွင် စာသားဖန်တီးရန် ကူညီပေးသည့် ရိုးရှင်းသော generative models များကို အဓိကထား၍ လေ့လာသွားမည်ဖြစ်သည်။ ရိုးရှင်းစွာပြောရမည်ဆိုပါက **character-level network** တစ်ခုကို တည်ဆောက်မည်ဖြစ်ပြီး၊ ၎င်းသည် စာလုံးတစ်လုံးချင်းစီဖြင့် စာသားကို ဖန်တီးပေးမည်ဖြစ်သည်။ သင်ကြားမှုအတွင်းတွင် စာသား corpus တစ်ခုကို ယူပြီး၊ ၎င်းကို စာလုံးအစီအစဉ်များအဖြစ် ခွဲထုတ်ရန် လိုအပ်မည်။


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## အက္ခရာအဆင့် စာပေဗဟုသုတ တည်ဆောက်ခြင်း

အက္ခရာအဆင့် ဖန်တီးမှုကွန်ရက်တစ်ခု တည်ဆောက်ရန်အတွက် စာသားကို စကားလုံးများမဟုတ်ဘဲ တစ်ခုချင်းစီ အက္ခရာများအဖြစ် ခွဲခြားရမည်။ ဒါကို အခြားသော tokenizer တစ်ခု သတ်မှတ်ခြင်းဖြင့် ပြုလုပ်နိုင်သည်။


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


ကျွန်တော်တို့ရဲ့ဒေတာစနစ်ကနေစာသားကိုဘယ်လိုကုဒ်ဖြစ်အောင်လုပ်နိုင်တယ်ဆိုတာကိုဥပမာနမူနာကြည့်ကြမယ်:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Generative RNN ကိုလေ့ကျင့်ခြင်း

RNN ကို စာသားထုတ်လုပ်ရန် လေ့ကျင့်ပုံမှာ အောက်ပါအတိုင်းဖြစ်ပါမည်။ အဆင့်တစ်ခုစီတွင် `nchars` အရှည်ရှိသော စာလုံးများ၏ အစဉ်အတိုင်းယူပြီး၊ နောက်ထွက်စာလုံးကို အဝင်စာလုံးတစ်ခုစီအတွက် ကွန်ယက်အားဖြင့် ထုတ်လုပ်ရန် တောင်းဆိုပါမည်။

![RNN သုံး၍ 'HELLO' စကားလုံးကို ထုတ်လုပ်နေသော ဥပမာပုံ။](../../../../../translated_images/my/rnn-generate.56c54afb52f9781d.webp)

အခြေအနေအလိုက်၊ *အဆုံးအမှတ်အသား* `<eos>` ကဲ့သို့သော အထူးအက္ခရာများကိုလည်း ထည့်သွင်းလိုတတ်ပါသည်။ သို့သော် ကျွန်ုပ်တို့အနေဖြင့် အဆုံးမရှိသော စာသားထုတ်လုပ်မှုအတွက် ကွန်ယက်ကိုသာ လေ့ကျင့်လိုသောကြောင့်၊ အစဉ်တစ်ခုစီ၏ အရွယ်အစားကို `nchars` အက္ခရာများအဖြစ် သတ်မှတ်ထားမည်ဖြစ်သည်။ ထို့ကြောင့်၊ လေ့ကျင့်မှုဥပမာတစ်ခုစီတွင် `nchars` အဝင်များနှင့် `nchars` အထွက်များ (အဝင်အစဉ်ကို ဘယ်ဘက်သို့ အက္ခရာတစ်လုံးရွှေ့ထားသောအတိုင်း) ပါဝင်မည်ဖြစ်သည်။ Minibatch တစ်ခုတွင် ဤအစဉ်များစွာ ပါဝင်မည်ဖြစ်သည်။

Minibatch များကို ထုတ်လုပ်ပုံမှာ `l` အရှည်ရှိသော သတင်းစာသားတစ်ခုစီကို ယူပြီး၊ ထိုစာသားမှ အဝင်-အထွက် အစဉ်အတိုင်းဖြစ်နိုင်သမျှအားလုံးကို ထုတ်ယူမည်ဖြစ်သည် (ထိုအစဉ်များမှာ `l-nchars` ဖြစ်မည်)။ ၎င်းတို့သည် minibatch တစ်ခုကို ဖွဲ့စည်းမည်ဖြစ်ပြီး၊ လေ့ကျင့်မှုအဆင့်တစ်ခုစီတွင် minibatch များ၏ အရွယ်အစားမှာ မတူကွဲပြားနေမည်ဖြစ်သည်။


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

ယခု Generator Network ကို သတ်မှတ်ကြမယ်။ ဒါကို ယခင်ယူနစ်မှာ ဆွေးနွေးခဲ့တဲ့ Recurrent Cell (simple, LSTM, GRU) မည်သည့်အရာကိုမဆို အခြေခံပြီး ဖန်တီးနိုင်ပါတယ်။ ဥပမာအနေနဲ့ ကျွန်တော်တို့ LSTM ကို အသုံးပြုပါမယ်။

Network က စာလုံးတွေကို input အနေနဲ့ လက်ခံရမှာဖြစ်ပြီး၊ Vocabulary size ကလည်း သေးငယ်တဲ့အတွက် Embedding Layer မလိုအပ်ပါဘူး၊ One-hot-encoded input ကို တိုက်ရိုက် LSTM cell ထဲသို့ ပို့နိုင်ပါတယ်။ သို့သော် စာလုံးနံပါတ်တွေကို input အနေနဲ့ ပေးရတဲ့အတွက် LSTM ထဲသို့ ပို့မီ One-hot-encode လုပ်ရပါမယ်။ ဒါကို `forward` pass အတွင်း `one_hot` function ကို ခေါ်သုံးပြီး လုပ်ဆောင်ပါတယ်။ Output Encoder က Hidden State ကို One-hot-encoded output အဖြစ် ပြောင်းလဲပေးမယ့် Linear Layer ဖြစ်ပါမယ်။


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

လေ့ကျင့်မှုအတွင်းမှာ ကျွန်တော်တို့ ဖန်တီးထားတဲ့ စာသားကို နမူနာယူနိုင်ဖို့ လိုအပ်ပါတယ်။ အဲဒီအတွက် `generate` ဆိုတဲ့ function ကို သတ်မှတ်ပြီး `size` အရှည်ရှိတဲ့ output string ကို စတင် string `start` ကနေ စတင်ဖန်တီးမှာ ဖြစ်ပါတယ်။

အလုပ်လုပ်ပုံက အောက်ပါအတိုင်း ဖြစ်ပါတယ်။ ပထမဦးဆုံးမှာ start string အားလုံးကို network မှတဆင့် ဖြတ်သွားပြီး output state `s` နဲ့ နောက်ထပ်ခန့်မှန်းထားတဲ့ အက္ခရာ `out` ကို ရယူပါမယ်။ `out` က one-hot encoded ဖြစ်တဲ့အတွက် `argmax` ကို သုံးပြီး vocabulary ထဲမှာ အက္ခရာ `nc` ရဲ့ index ကို ရယူပါမယ်။ ထို့နောက် `itos` ကို သုံးပြီး အမှန်တကယ် အက္ခရာကို ရှာဖွေပြီး အဲဒီအက္ခရာကို `chars` ဆိုတဲ့ အက္ခရာများရဲ့ list ထဲမှာ ထည့်သွင်းပါမယ်။ အက္ခရာတစ်ခုကို ဖန်တီးတဲ့ ဒီလုပ်ငန်းစဉ်ကို `size` ကြိမ် ထပ်မံလုပ်ဆောင်ပြီး လိုအပ်တဲ့ အက္ခရာအရေအတွက်ကို ဖန်တီးပါမယ်။


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

အခုတော့ လေ့ကျင့်မှုကို စတင်ကြမယ်! လေ့ကျင့်မှု loop က အရင်က ဥပမာတွေမှာလိုပဲ ဖြစ်ပေမယ့် accuracy အစား 1000 epochs တစ်ခါစီမှာ ထုတ်လုပ်ထားတဲ့ စမ်းသပ်စာသားကို ပုံနှိပ်ပြသမှာ ဖြစ်ပါတယ်။

Loss ကိုတွက်ချက်တဲ့နည်းလမ်းကို အထူးဂရုစိုက်ဖို့ လိုအပ်ပါတယ်။ Loss ကို တစ်ခုချင်းစီ encode လုပ်ထားတဲ့ output `out` နဲ့ မျှော်မှန်းထားတဲ့စာသား `text_out` (character indices စာရင်း) ကို အသုံးပြုပြီးတွက်ချက်ရမယ်။ ကံကောင်းစွာဖြင့် `cross_entropy` function က unnormalized network output ကို ပထမ argument အနေနဲ့၊ class number ကို ဒုတိယ argument အနေနဲ့ လိုအပ်တယ်၊ ဒါက ကျွန်တော်တို့မှာရှိပြီးသားပဲ ဖြစ်ပါတယ်။ ဒါ့အပြင် minibatch size အပေါ်မှာ အလိုအလျောက် အလယ်ပျံ့တွက်ချက်မှုကိုလည်း လုပ်ဆောင်ပေးပါတယ်။

လေ့ကျင့်မှုကို `samples_to_train` sample အရေအတွက်နဲ့ ကန့်သတ်ထားပြီး အချိန်မကြာအောင် လုပ်ဆောင်ထားပါတယ်။ သင်တို့ကို အတတ်နိုင်ဆုံး လေ့လာမှုကို စမ်းသပ်ဖို့ တိုက်တွန်းပါတယ်၊ အချို့သော epoch အတွက် အချိန်ပိုကြာတဲ့ လေ့ကျင့်မှုကို စမ်းသပ်ကြည့်ပါ (ဒီအခါမှာ ဒီ code အပေါ်မှာ loop တစ်ခုထပ်ထည့်ဖွဲ့စည်းဖို့ လိုအပ်ပါလိမ့်မယ်)။


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

ဤဥပမာသည် အတော်လေးကောင်းမွန်သောစာသားကို ရရှိစေပြီးဖြစ်သော်လည်း အချို့အပိုင်းများကို ပိုမိုတိုးတက်စေရန် အခြားနည်းလမ်းများလည်း ရှိပါသည် -

* **ပိုမိုကောင်းမွန်သော minibatch ဖန်တီးမှု**။ ကျွန်ုပ်တို့သည် သင်ကြားမှုအတွက် ဒေတာကို ပြင်ဆင်သည့်နည်းလမ်းမှာ တစ်ခုတည်းသောနမူနာမှ minibatch တစ်ခုကို ဖန်တီးခြင်းဖြစ်သည်။ ဒါဟာ အကောင်းဆုံးနည်းလမ်းမဟုတ်ပါ၊ အကြောင်းမှာ minibatch များသည် အရွယ်အစားကွဲပြားမှုရှိပြီး၊ တချို့ minibatch များကိုတော့ ဖန်တီးလို့မရနိုင်ပါ၊ အကြောင်းမှာ စာသားသည် `nchars` ထက် သေးငယ်နေခြင်းကြောင့်ဖြစ်သည်။ ထို့အပြင် သေးငယ်သော minibatch များသည် GPU ကို လုံလောက်စွာ အသုံးမပြုနိုင်ပါ။ အကောင်းဆုံးနည်းလမ်းမှာ နမူနာအားလုံးမှ စာသားတစ်ခုလုံးကို စုစည်းပြီး၊ input-output အစုံအားလုံးကို ဖန်တီးပြီး၊ shuffle ပြုလုပ်ပြီး၊ အရွယ်အစားတူညီသော minibatch များကို ဖန်တီးရန်ဖြစ်သည်။

* **Multilayer LSTM**။ LSTM cell များကို အလွှာ ၂ သို့မဟုတ် ၃ အထိ စမ်းသပ်ကြည့်ရန် make sense ဖြစ်ပါသည်။ ယခင်ယူနစ်တွင် ပြောခဲ့သည့်အတိုင်း၊ LSTM ၏ အလွှာတစ်ခုစီသည် စာသားမှ အချို့သော pattern များကို ထုတ်ယူပေးပါသည်။ Character-level generator ၏ အနေဖြင့် LSTM ၏ အနိမ့်ဆုံးအလွှာသည် syllable များကို ထုတ်ယူရန် တာဝန်ရှိမည်ဟု မျှော်လင့်ရပြီး၊ အထက်ဆုံးအလွှာများသည် စကားလုံးများနှင့် စကားလုံးပေါင်းများကို ထုတ်ယူရန် တာဝန်ရှိမည်ဖြစ်သည်။ ၎င်းကို LSTM constructor သို့ အလွှာအရေအတွက် parameter ကို ပေးပို့ခြင်းဖြင့် ရိုးရှင်းစွာ အကောင်အထည်ဖော်နိုင်ပါသည်။

* သင်သည် **GRU unit များ** ကိုလည်း စမ်းသပ်ကြည့်လိုနိုင်ပြီး၊ **hidden layer အရွယ်အစားကွဲပြားမှုများ** ကိုလည်း စမ်းသပ်ကြည့်နိုင်ပါသည်။ Hidden layer အရွယ်အစားသည် အလွန်ကြီးလွန်းပါက overfitting ဖြစ်စေနိုင်ပြီး (ဥပမာ - network သည် စာသားကို တိတိကျကျ သင်ယူသွားမည်ဖြစ်သည်)၊ သေးငယ်လွန်းပါက ကောင်းမွန်သောရလဒ် မရနိုင်ပါ။


## Soft text generation and temperature

ယခင် `generate` ကိုဖော်ပြထားသော အဓိပ္ပါယ်အရ၊ ဖန်တီးထားသော စာသားတွင် နောက်တစ်ခုထည့်ရန်အတွက် အမြင့်ဆုံး probability ရှိသော အက္ခရာကို အမြဲယူခဲ့သည်။ ဒါကြောင့် စာသားသည် အက္ခရာအဆင့်များကို ထပ်ခါတလဲလဲ ပြန်လည်အသုံးပြုနေသော အခြေအနေဖြစ်လာခဲ့သည်။ ဥပမာအားဖြင့်:
```
today of the second the company and a second the company ...
```

သို့သော် နောက်ထပ် အက္ခရာအတွက် probability distribution ကိုကြည့်မယ်ဆိုရင်၊ အမြင့်ဆုံး probability ရှိသော အက္ခရာများအကြား ကွာခြားမှုသည် များစွာမရှိနိုင်ပါ။ ဥပမာအားဖြင့် အက္ခရာတစ်ခုမှာ probability 0.2 ရှိနိုင်ပြီး၊ နောက်တစ်ခုမှာ 0.19 ရှိနိုင်သည်။ ဥပမာအားဖြင့် '*play*' ဆိုသော စာကြောင်းတွင် နောက်ထပ် အက္ခရာသည် space ဖြစ်နိုင်သလို၊ **e** (ဥပမာအားဖြင့် *player* စကားလုံးတွင်) ဖြစ်နိုင်ပါသည်။

ဒါကြောင့် အမြင့်ဆုံး probability ရှိသော အက္ခရာကို ရွေးချယ်ခြင်းသည် အမြဲတမ်း "တရားမျှတ" မဖြစ်နိုင်ကြောင်း သင်ခန့်မှန်းနိုင်ပါသည်။ ဒါကြောင့် ဒုတိယမြင့်ဆုံး probability ရှိသော အက္ခရာကို ရွေးချယ်ခြင်းသည်လည်း အဓိပ္ပါယ်ရှိသော စာသားကို ဖန်တီးနိုင်စေပါသည်။ ထို့ကြောင့် network output မှပေးသော probability distribution အရ **sample** လုပ်ခြင်းသည် ပိုမိုဉာဏ်ရှိသော နည်းလမ်းဖြစ်သည်။

ဒီ sampling ကို **multinomial distribution** ဟုခေါ်သော နည်းလမ်းကို အသုံးပြုသော `multinomial` function ကို အသုံးပြု၍ ပြုလုပ်နိုင်ပါသည်။ **soft** text generation ကို အကောင်အထည်ဖော်သော function ကို အောက်တွင် ဖော်ပြထားပါသည်:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

ကျွန်ုပ်တို့သည် **အပူချိန်** ဟုခေါ်သော နောက်ထပ် parameter တစ်ခုကို မိတ်ဆက်ပေးခဲ့ပြီး၊ ၎င်းသည် အမြင့်ဆုံး probability ကို များစွာလိုက်နာသင့်ကြောင်းကို ဖော်ပြရန် အသုံးပြုသည်။ အပူချိန်သည် 1.0 ဖြစ်ပါက၊ fair multinomial sampling ကို ပြုလုပ်ပြီး၊ အပူချိန်သည် အဆုံးမရှိအထိ မြင့်တက်သွားသောအခါ - probability အားလုံးသည် တူညီသွားပြီး၊ နောက် character ကို အလွတ်ရွေးချယ်သည်။ အောက်ပါ ဥပမာတွင် အပူချိန်ကို များစွာမြှင့်တင်သည့်အခါ၊ စာသားသည် အဓိပ္ပါယ်မရှိသွားကြောင်းကို တွေ့နိုင်ပြီး၊ အပူချိန်သည် 0 အနီးသို့ ရောက်လာသောအခါ "cycled" အခက်အခဲဖြင့် ဖန်တီးထားသော စာသားနှင့် ဆင်တူသွားသည်ကို တွေ့နိုင်သည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
